In [1]:
import pandas as pd 
import numpy as np
import pandas as pd 
import numpy as np
import torch
import csv
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import re
from bs4 import BeautifulSoup
import contractions
import re
import unicodedata
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from transformers import BertTokenizer
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm  # For a nice progress bar!
from torch.utils.data import Dataset, DataLoader

from os import listdir
from os.path import isfile, join

In [6]:
path = '/Users/tushar/iisc/sem3/TSE/project/Why-Will-My-Question-Be-Closed/Dataset/sample_0.02_33.csv'
df = pd.read_csv(path,dtype={'OwnerUserId': 'float64'})

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
# Keeping only Questions i.e removing Answers, wiki, comments etc
df = df[ df['PostTypeId'] == 1]  

In [42]:
# Making new column to represent if question is closed or not 
df['closed'] = df['ClosedDate'].notnull().astype(int)

/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/400001917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['closed'] = df['ClosedDate'].notnull().astype(int)


In [43]:

df['closed'].value_counts()

1    775744
0    318888
Name: closed, dtype: int64

In [44]:
df['ClosedDate'] = pd.to_datetime( df['ClosedDate'])
df['CreationDate'] = pd.to_datetime( df['CreationDate'])

/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/2306763115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ClosedDate'] = pd.to_datetime( df['ClosedDate'])
/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/2306763115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CreationDate'] = pd.to_datetime( df['CreationDate'])


In [45]:
# Filtering datasets

startDate = '07-01-2013'
df = df[df['CreationDate'] >= startDate ] 


In [46]:
df['closed'].value_counts()

1    775744
0    318888
Name: closed, dtype: int64

In [47]:
closed_reason_df = pd.read_csv('/Users/tushar/iisc/sem3/TSE/project/ClosedPosts-Type/closed_reasons.csv')
closed_questions =df[ df['closed'] == 1 ]
open_questions = df[ df['closed'] == 0 ]

In [48]:
closed_reason_df.head(10)

,RowID,id,comment
0,1,17396037,101
1,2,17396048,104
2,3,17396074,103
3,4,17396115,102
4,5,17396139,102
5,6,17396346,101
6,7,17396354,102
7,8,17396390,102
8,9,17396418,103
9,10,17396469,105


In [49]:
open_questions['comment'] = 0 

,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed,comment
281078,4453,17398716,1,How and where do I handle file uploads from dj...,<django>,<p>So ive read over the docs and I have come o...,2013-07-01 06:41:32.790,NaT,0,0
281080,3365,17397251,1,Why is my view black?,<iphone><ios><objective-c><core-graphics><cgrect>,<p>My view is a black rectangle...but it shoul...,2013-07-01 04:00:03.057,NaT,0,0
281082,5290,17399806,1,OutOfMemoryError during BitmapFactory.decodeBy...,<android><android-camera><out-of-memory>,<p>I'm getting an <code>OutOfMemoryError</code...,2013-07-01 08:02:07.390,NaT,0,0
281084,6299,17401154,1,Android NFC java.io.IOException: Transceive fa...,<android><exception><nfc>,<p>I am working on an android application whic...,2013-07-01 09:27:04.517,NaT,0,0
281087,8163,17403551,1,OpenGL UI library for C# with OpenTK,<c#><opengl><user-interface><opentk>,<p>I'm writing a game engine in C# using OpenT...,2013-07-01 11:36:17.700,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...
1059143,447,69059810,1,"My for-loop gets stuck, but doesn't exit out o...",<java><for-loop>,<p>Here is the code. It gets stuck in the for-...,2021-09-05 00:33:06.447,NaT,0,0
1059149,1132,69060537,1,How to implement uipath picture in picture fea...,<windows><containers><rpa>,<p>How to implement uipath picture in picture ...,2021-09-05 04:22:42.837,NaT,0,0
1059150,964,69060363,1,Flutter: Is there a cleaner way to infinite sc...,<flutter><google-cloud-firestore><flutter-prov...,<p>Problem: I'm trying to implement an infinit...,2021-09-05 03:24:55.067,NaT,0,0
1059151,1120,69060525,1,Why does it say Line 31 Invalid expression Whe...,<matlab>,<p>fprintf('Thermal energy removed from the ma...,2021-09-05 04:19:21.740,NaT,0,0


In [56]:
# left joining dataset with closed question dataset to obtain reason for closing 
merged_closed_df = pd.merge(closed_questions,closed_reason_df,left_on = 'Id',right_on = 'id')

In [58]:
len(merged_closed_df)

772378

In [31]:
# No need to do this now 

# Removing posts which were closed in past but now open and by-mistakenly added in closed_reason_df

merged_df = (merged_df[~((merged_df['comment'].notnull()) & (merged_df['closed'] == 0 ))])

In [59]:
# Removing duplicate closed questions 
merged_closed_df = merged_closed_df[~(merged_closed_df['comment'] == 101.0)]

In [62]:
# Replaing values with proper class labels 
merged_closed_df['comment'] = merged_closed_df['comment'].replace(np.nan,0)
merged_closed_df['comment'] = merged_closed_df['comment'].replace(102.0,1)
merged_closed_df['comment'] = merged_closed_df['comment'].replace(103.0,2)
merged_closed_df['comment'] = merged_closed_df['comment'].replace(104.0,3)
merged_closed_df['comment'] = merged_closed_df['comment'].replace(105.0,4)

/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/2199794189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_closed_df['comment'] = merged_closed_df['comment'].replace(np.nan,0)
/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/2199794189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_closed_df['comment'] = merged_closed_df['comment'].replace(102.0,1)
/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/2199794189.py:4: SettingWithCopyWarning:

In [64]:
# converting to ints
merged_closed_df['comment'] = merged_closed_df['comment'].astype(int)


/var/folders/sb/zmq5ftc11rx8fjgq0s5g09z40000gn/T/ipykernel_70749/4051967042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_closed_df['comment'] = merged_closed_df['comment'].astype(int)


In [67]:
len(merged_closed_df)

265163

In [35]:
# Generating final dataset
merged_df_no_nans = merged_df.replace(np.nan,"") 

In [36]:
# Keeping only Questions i.e removing Answers, wiki, comments etc
merged_df_no_nans = merged_df_no_nans[ merged_df_no_nans['PostTypeId'] == 1 ]

In [37]:
merged_df_no_nans['closed'].value_counts()

1.0    265163
Name: closed, dtype: int64

In [38]:
merged_df_no_nans 

,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed,RowID,id,comment
1,2383.0,17396048.0,1.0,"Apply focus to one object, after 10 seconds ap...",<javascript><settimeout>,<p>I want to perform a set timeout function wi...,2013-07-01 00:20:07.363,2013-07-01 06:01:55.213,1.0,2,17396048,3
2,2406.0,17396074.0,1.0,what's wrong with this code? I have just start...,<python>,"<pre><code>print ""Welcome to the English to Pi...",2013-07-01 00:23:52.207,2013-07-01 00:29:19.837,1.0,3,17396074,2
3,2441.0,17396115.0,1.0,Fill out web page template with form data,<php><forms><templates><dynamic><placeholder>,<p>I have a webpage template already designed ...,2013-07-01 00:31:56.723,2013-07-01 02:25:41.440,1.0,4,17396115,1
4,2463.0,17396139.0,1.0,The public link of dropbox doestn't work,<dropbox>,"<p>Recently, I find the public link of dropbox...",2013-07-01 00:37:14.973,2013-07-01 02:25:55.520,1.0,5,17396139,1
7,2671.0,17396390.0,1.0,how do i save a winform to pdf using the save ...,<vb.net>,<p>I have a winform with a menu strip and an o...,2013-07-01 01:30:58.907,2014-03-04 18:56:59.237,1.0,8,17396390,1
...,...,...,...,...,...,...,...,...,...,...,...,...
827191,5400.0,69058411.0,1.0,Using __init__ method to call class main funct...,<python>,<p>I needed to encapsulate functions related t...,2021-09-04 19:33:59.397,2021-09-04 19:38:15.220,1.0,827192,69058411,4
827204,5786.0,69058843.0,1.0,Why Can I only add a further-in-the-list Codew...,<python><list><variables><index-error>,"<p>I am making a decoder, but when I type a co...",2021-09-04 20:43:27.100,2021-09-04 21:49:03.650,1.0,827205,69058843,1
827210,6217.0,69059316.0,1.0,TypeError: unsupported operand type(s) for /: ...,<python><machine-learning><cross-validation><m...,<pre><code> x = df[header] &#xA; clf=Gau...,2021-09-04 22:23:54.347,2021-09-05 00:41:20.350,1.0,827211,69059316,1
827229,584.0,69059952.0,1.0,How do I display an option from a <select> tag...,<javascript><html>,<p>So I am trying to make a simple budget app ...,2021-09-05 01:15:50.383,2021-09-05 05:02:33.530,1.0,827230,69059952,1


In [39]:
open_from_posts =df[ df['closed'] == 0 ]


,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed
281072,2678,17396401,2,NaN,NaN,<p>Remove the class <code>date</code></p>&#xA;...,2013-07-01 01:33:05.127,NaT,0
281073,6912,17401974,2,NaN,NaN,<p>One can also obtain the characters from <co...,2013-07-01 10:13:33.500,NaT,0
281074,3355,17397241,2,NaN,NaN,"<p>For most of the time, it is more desirable ...",2013-07-01 03:58:39.507,NaT,0
281075,4036,17398162,2,NaN,NaN,<p>Assuming that the distance counts only swap...,2013-07-01 05:53:15.767,NaT,0
281076,6144,17400950,2,NaN,NaN,"<p>You're right, Play Framework automatically ...",2013-07-01 09:16:13.340,NaT,0
281077,6131,17400933,2,NaN,NaN,"<pre><code>private static Map&lt;String, Strin...",2013-07-01 09:15:06.507,NaT,0
281078,4453,17398716,1,How and where do I handle file uploads from dj...,<django>,<p>So ive read over the docs and I have come o...,2013-07-01 06:41:32.790,NaT,0
281079,6596,17401541,2,NaN,NaN,<p>Use this:</p>&#xA;&#xA;<pre><code>echo arra...,2013-07-01 09:49:50.130,NaT,0
281080,3365,17397251,1,Why is my view black?,<iphone><ios><objective-c><core-graphics><cgrect>,<p>My view is a black rectangle...but it shoul...,2013-07-01 04:00:03.057,NaT,0
281081,4494,17398769,2,NaN,NaN,<p>You will have to use javascript. Start by g...,2013-07-01 06:46:19.937,NaT,0


In [22]:
final_df = merged_df_no_nans

In [26]:
final_df[ (final_df['closed'] == 1) & (final_df['comment'] == 0 ) ]

,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed,RowID,id,comment
780888,8106,17566867,1,Change drive letter using WinDbg,<windows><windbg><remote-debugging>,<p>Can I change the label of USB connected to ...,2013-07-10 09:19:53.310,2013-07-10 11:13:22.313,1,,,0
782217,3703,17646488,1,How do I know the SIP has completed the call?,<sip><voip><asterisk><freeswitch>,<p>How do I know the SIP has completed the cal...,2013-07-15 03:49:57.690,2013-12-17 04:32:51.730,1,,,0
782284,7195,17650862,1,PHP script returning data,<php><sql><localhost>,<p>I have downloaded this script from web in o...,2013-07-15 09:27:40.960,2013-08-17 18:25:25.207,1,,,0
782420,5170,17658961,1,Advanced html tables with Knitr in R - alterna...,<html><css><r><knitr>,<p>When it comes to html out generation with K...,2013-07-15 16:19:51.147,2013-09-08 23:09:09.517,1,,,0
791041,7244,18204566,1,Binary file Permission denied,<linux><asterisk><permission-denied>,<p>I work with asterisk 11.2.1 and I try to us...,2013-08-13 08:48:12.933,2013-08-13 11:01:46.217,1,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1558923,1312,69060726,1,how to write this code without using any funct...,<c++>,<p>/*i have a project so far i thin im doing g...,2021-09-05 05:21:31.253,2021-09-05 05:40:39.177,1,,,0
1558925,1357,69060772,1,How can extract a hidden product link of a sto...,<web><web-scraping><bots>,<p><strong>Store page example:</strong> <a hre...,2021-09-05 05:30:57.353,2021-09-05 06:20:32.367,1,,,0
1558927,1443,69060859,1,"What does "">location.href"" Script does here?",<javascript><html>,<p>So I have this Spam/adware/phishing link an...,2021-09-05 05:48:08.093,2021-09-05 05:51:09.543,1,,,0
1558928,1471,69060891,1,What is the least case Binary function?,<algorithm><binary-search>,<p>I have learnt about binary search.&#xA;Let'...,2021-09-05 05:57:52.653,2021-09-05 06:20:15.327,1,,,0


In [28]:
final_df

,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed,RowID,id,comment
6,4453,17398716,1,How and where do I handle file uploads from dj...,<django>,<p>So ive read over the docs and I have come o...,2013-07-01 06:41:32.790,NaT,0,,,0
8,3365,17397251,1,Why is my view black?,<iphone><ios><objective-c><core-graphics><cgrect>,<p>My view is a black rectangle...but it shoul...,2013-07-01 04:00:03.057,NaT,0,,,0
10,5290,17399806,1,OutOfMemoryError during BitmapFactory.decodeBy...,<android><android-camera><out-of-memory>,<p>I'm getting an <code>OutOfMemoryError</code...,2013-07-01 08:02:07.390,NaT,0,,,0
12,6299,17401154,1,Android NFC java.io.IOException: Transceive fa...,<android><exception><nfc>,<p>I am working on an android application whic...,2013-07-01 09:27:04.517,NaT,0,,,0
14,8163,17403551,1,OpenGL UI library for C# with OpenTK,<c#><opengl><user-interface><opentk>,<p>I'm writing a game engine in C# using OpenT...,2013-07-01 11:36:17.700,NaT,0,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1558923,1312,69060726,1,how to write this code without using any funct...,<c++>,<p>/*i have a project so far i thin im doing g...,2021-09-05 05:21:31.253,2021-09-05 05:40:39.177,1,,,0
1558925,1357,69060772,1,How can extract a hidden product link of a sto...,<web><web-scraping><bots>,<p><strong>Store page example:</strong> <a hre...,2021-09-05 05:30:57.353,2021-09-05 06:20:32.367,1,,,0
1558927,1443,69060859,1,"What does "">location.href"" Script does here?",<javascript><html>,<p>So I have this Spam/adware/phishing link an...,2021-09-05 05:48:08.093,2021-09-05 05:51:09.543,1,,,0
1558928,1471,69060891,1,What is the least case Binary function?,<algorithm><binary-search>,<p>I have learnt about binary search.&#xA;Let'...,2021-09-05 05:57:52.653,2021-09-05 06:20:15.327,1,,,0


In [89]:
final_df['comment'].value_counts()

0    401413
1    121387
2     63013
3     59072
4     21691
Name: comment, dtype: int64

In [98]:
'''
checking how many closed questions were generated during march 2019 till now. 
because author mentioned that 207,040 questions from June 2013 to March 2019
'''

len(final_df[final_df['closed'] == 1]) - len(final_df[(final_df['closed'] == 1 ) & (final_df['CreationDate'] >= '05-03-2019') ])

261615

# We have 330752 closed questions and 335824 open questions to train on 


In [99]:
final_df.to_csv('final_df.csv')

In [82]:
merged_df_no_nans['closed'].value_counts()

0    335824
1    330752
Name: closed, dtype: int64

In [49]:
# df["Title"] = df["Title"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://localhost:8080/webpack-dev-server/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [55]:
# df["Body"] = df["Body"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))
# df["Tags"] = df["Tags"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.nuget.org/packages/FakeDbSet" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


### Trying Vainilla BERT Tokenizer and RNN 

In [7]:
# Encode the sentence
tz = BertTokenizer.from_pretrained("bert-base-cased")
def tokenize(df,max_length):
    input_ids = []
    for sent in df:
        encoded = tz.encode_plus(
            text=sent,  # the sentence to be encoded
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length = max_length,  # maximum length of a sentence
            pad_to_max_length=True,  # Add [PAD]s
            return_attention_mask = True,  # Generate the attention mask
            truncation=True
        )
        # Get the input IDs and attention mask in tensor format
        input_ids.append(encoded['input_ids'])
        
        
    return torch.tensor(input_ids)
        
    

In [91]:
df['Tags'].str.len().describe()


count    884151.000000
mean         29.624135
std          15.561090
min           3.000000
25%          18.000000
50%          28.000000
75%          39.000000
max         126.000000
Name: Tags, dtype: float64

In [8]:
title_max_length = 150
body_max_length = 1500 
tags_max_length = 150 
df.replace(np.nan, '', inplace=True)
title_tensor = tokenize(df['Title'],title_max_length)
body_tensor = tokenize(df['Body'],body_max_length)
tags_tensor = tokenize(df['Tags'],tags_max_length)

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
torch.save(title_tensor, 'title_tensor.pt')
torch.save(body_tensor, 'body_tensor.pt')
torch.save(tags_tensor, 'tags_tensor.pt')

In [24]:
torch.save(tz.vocab,'vocab.v')

In [39]:
len(df['closed'])

2111792

In [50]:
class CustomTextDataset(Dataset):
    def __init__(self, txt, labels):
        self.labels = labels
        self.text = txt
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
#         sample = {"Text": text, "Class": label}
        return (text,label)

# define data set object
dataset = CustomTextDataset(title_tensor,df['closed'].to_numpy())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [77]:

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 256
hidden_size = 256
num_layers = 2
num_classes = 2
sequence_length = title_tensor.size(1)
learning_rate = 0.005
batch_size = 2048
num_epochs = 3

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes,num_embeddings,embedding_size = 256):
        super(RNN, self).__init__()
        self.emb = nn.Embedding(num_embeddings,embedding_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        
        x = self.emb(x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out



In [78]:
# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = RNN(input_size, hidden_size, num_layers, num_classes,len(tz.vocab)).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

        

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 825/825 [47:36<00:00,  3.46s/it]


Accuracy on training set: 98.035255
Accuracy on test set: 98.00


In [79]:
torch.save(model)

TypeError: save() missing 1 required positional argument: 'f'

In [81]:
torch.save(model,"RNN_title_0.model")